In [ ]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
table = pd.read_excel('parser/tables/drug_nodes_identifiers.xlsx')

In [ ]:
drug_id = table["drugbank-id"]

In [ ]:
def extract_words_between_strings(sentence, start_string, end_string):
    extracted_words = []
    start_index = sentence.find(start_string)

    while start_index != -1:
        start_index += len(start_string)
        end_index = sentence.find(end_string, start_index)

        if end_index != -1:
            words = sentence[start_index:end_index].strip()
            extracted_words.append(words)
            start_index = sentence.find(start_string, end_index)
        else:
            break

    return extracted_words

In [ ]:
start_string = "Associated Conditions"
end_string = "Contraindications"
baseURL = "https://go.drugbank.com/drugs/"

drug_cond = pd.DataFrame()

In [ ]:
for dbid in drug_id:
  page = req.get(baseURL+str(dbid))
  soup = BeautifulSoup(page.content, "html.parser")
  content = soup.text
  # print(content)
  disease_list = extract_words_between_strings(content, start_string, end_string)
  disease_list = [disease for disease in disease_list if disease.strip()]
  disease_names = []
  for disease in disease_list:
      disease_names.extend(disease.split('\n'))
  disease_names = [name.strip() for name in disease_names]
  data = pd.DataFrame({"assoc_cond": disease_names})
  data["drugbank-id"] = dbid
  drug_cond = drug_cond.append(data).reset_index(drop=True)

In [ ]:
df_filtered = drug_cond[drug_cond['assoc_cond'] != "Associated Therapies"]
df_filtered = df_filtered[df_filtered['assoc_cond'] != ""]

In [ ]:
df_filtered['assoc_cond'] = df_filtered['assoc_cond'].str.replace(r'(.*)Associated Therapies.*', r'\1', regex=True)

In [ ]:
df_filtered

,assoc_cond,drugbank-id
0,Locally Advanced Squamous Cell Carcinomas of t...,DB00002
1,Metastatic Colorectal Cancer (CRC),DB00002
2,Metastatic Squamous Cell Carcinoma of the Head...,DB00002
3,Regionally Advanced Squamous Cell Carcinoma of...,DB00002
4,Recurrent Squamous Cell Carcinoma of the Head ...,DB00002
...,...,...
15006,Coronavirus Disease 2019 (COVID‑19),DB17090
15007,Coronavirus Disease 2019 (COVID‑19),DB17091
15008,Coronavirus Disease 2019 (COVID‑19),DB17095
15009,High risk BCG-unresponsive non-muscle invasive...,DB17381


No charts were generated by quickchart


In [ ]:
df_filtered.to_csv("DBID_AssocCondn.csv", drop_index=True)